In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from os import path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

## Load data
Load interpolated/point data for vessels/whales - this is at 10 minute resolution, allowing detection of encounters between vessels/whales

In [ ]:
from data_processing import load_data

vessel_data_files = [path.join('data', 'vessels', f'{vessel_type}_points.gpkg') for vessel_type in ['Fishing', 'Other', 'Cargo', 'Passenger', 'Tanker']]
vessel_data_sets = [load_data.load_vessel_points(vdf, 2193) for vdf in vessel_data_files]

vessel_points = pd.concat(vessel_data_sets)

In [ ]:
whales_interp = load_data.load_whales('data/whales/df_all_3.csv', vessel_points.geometry.total_bounds, 2193, 10)

In [ ]:
from data_processing import encounters

fname = 'encounters.pickle'
if path.isfile(fname):
    vessel_points = pd.read_pickle(fname)
else:
    vessel_points['whale_dist'] = encounters.dist_to_whales(vessel_points, whales_interp)
    vessel_points.to_pickle('encounters.pickle')

In [ ]:
whales, vessels, protected_areas, basemap, bounds = load_data.load_all()

In [ ]:
whales_lines = load_data.load_whale_lines('data/whales/df_all_3.csv', 2193)

In [ ]:
# Simplify geometry & drop columns to reduce html size

whales_lines.drop(columns=['id'], inplace=True)
vessel_points.drop(columns=['timestamp', 'callsign', 'type'], inplace=True)

## Plotting
Plot vessel + whale traces, color encounters

In [ ]:
from plotting import heatmap

In [ ]:
from bokeh.io import output_notebook, output_file, export_png
from bokeh.plotting import show
output_notebook()
output_file('encounters.html', title='Encounters')

In [ ]:
p = heatmap.encounters_map(whales_lines, vessels, vessel_points, protected_areas, basemap, bounds, max_dist=20000)
show(p)

In [ ]:
export_png(p, filename='encounters.png')